## Acoustic Integration - Basic Forward Rendering

In [ ]:
from tqdm import trange

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import mitsuba as mi

from utils import drjit_turn_off_optimizations, shoebox_scene, shoebox_scene_visual, plot_img
from acoustic_params import EDC

plt.style.use('ggplot')

mi.set_variant('cuda_ad_acoustic')
mi.set_log_level(mi.LogLevel.Warn)

drjit_turn_off_optimizations(False)

### 25x12x7 Box

In [ ]:
mi.set_variant('cuda_ad_acoustic')

absorption = [0.1, 0.25, 0.5, 0.75, 0.9]
scattering = [0.0, 0.25, 0.5, 0.75, 1.0] #* 2

absorption_mdim, scattering_mdim = np.meshgrid(absorption, scattering)
absorption,      scattering      = absorption_mdim.flatten(), scattering_mdim.flatten()

config = {
    "box_dim":        [25., 12., 7.],
    "mic_pos":        [ 9.,  6., 1.],
    "speaker_pos":    [20.,  7., 2.],
    "speaker_radius": 0.1,
    "interference":   False,

    "absorption": [(i + 1, a) for i, a in enumerate(absorption)],
    "scattering": [(i + 1, s) for i, s in enumerate(scattering)],

    "wav_bins":  scattering.shape[0], # x
    "time_bins": 4000,                # y
    "max_time":  4.,

    "integrator": "acousticpath",
    "max_depth":  150,
    "spp":        2**22,
}

scene_dict = shoebox_scene(**config)
#mi.xml.dict_to_xml(scene_dict, "mitsuba_acoustic.xml")

# mic_posB = [20., 2., 6.]
# scene_dict["sensor"]["microphoneB"] = {
#     "type": "microphone",
#     "to_world": mi.ScalarTransform4f.translate(np.array(mic_posB) - (np.array(config['box_dim']) / 2.)),
#     # "to_world": mi.ScalarTransform4f.translate(np.array(config["mic_pos"]) - (np.array(config['box_dim']) / 2.)),
# }

time = np.linspace(0., config["max_time"], config["time_bins"], endpoint=False)
scene = mi.load_dict(scene_dict)

In [ ]:
data = np.array(mi.render(scene, seed=0))
hist = data[:, :, 0] / config["spp"]
cnts = data[:, :, 1]
edc  = EDC(hist)

In [ ]:
# compoare absorption, scattering fix

label = list(map(lambda x: f"$\\alpha = {x:.2f}$", absorption_mdim[0]))

n = scattering_mdim.shape[1]

for i in range(scattering_mdim.shape[0]):
    continue
    k = i * n

    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, sharex=True, figsize=(18, 4))
    fig.suptitle(f"scattering = {scattering_mdim[i, 0]}")
    ax1.plot(time, hist[:, k:k+n], label=label)
    ax1.set_title("Histogram")
    ax1.legend()
    ax2.plot(time, edc[:, k:k+n])
    ax2.set_title("Energy [dB]")
    ax3.plot(time, cnts[:, k:k+n])
    ax3.set_title("Counts")
    plt.tight_layout()
    #plt.savefig(f"figures/{s}_count.png")
    plt.show()

In [ ]:
# compoare scattering, absorption fix

label = list(map(lambda x: f"s {x:.2f}", scattering_mdim[:, 0]))

for i in range(scattering_mdim.shape[1]):
    continue
    idx = np.arange(scattering_mdim.shape[0]) * scattering_mdim.shape[1] + i

    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, sharex=True, figsize=(18, 4))
    fig.suptitle(f"$\\alpha = {absorption_mdim[0, 1]}$")
    ax1.plot(time, hist[:, idx], label=label)
    ax1.set_title("Histogram")
    ax1.legend()
    ax2.plot(time, edc[:, idx])
    ax2.set_title("Energy [dB]")
    ax3.plot(time, cnts[:, idx])
    ax3.set_title("Counts")
    plt.tight_layout()
    plt.show()

In [ ]:
# plot everything in 3d

if False:
    fig = plt.figure(figsize=(7, 7))
    ax  = fig.add_subplot(projection='3d')
    for i in range(hist.shape[1]):
        ax.plot(time, np.ones_like(time) * i, np.log(hist[:, i]))

    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_zlabel("z")

    plt.tight_layout()
    plt.show()

In [ ]:
# store simulation to disk

results = pd.DataFrame()

for i in range(scattering.shape[0]):
    s, a = scattering[i], absorption[i]
    ar, sr = round(a + 0.01, 1), round(s + 0.01, 1)
    results.insert(0, f'a{ar:.1f}-s{sr:.1f}', hist[:, i])

results.insert(0, "time", time)
results.set_index("time", inplace=True, drop=True)

# results.to_pickle('../data/mi3-prba-box-25-12-7.pkl')

### Variance

In [ ]:
data = np.zeros((50, config["time_bins"], config["wav_bins"]))
for i in trange(data.shape[0]):
    data[i] = np.array(mi.render(scene, seed=i))[:, :, 0]

In [ ]:
plt.plot(time, np.var(data[:, :, 10:15], axis=0))
plt.ylim(0, 20)
plt.show()

### Scene Visualization

In [ ]:
mi.set_variant('cuda_ad_rgb')
scene_dict = shoebox_scene_visual(resf=10, **config)

In [ ]:
tf          = mi.ScalarTransform4f
box_dim     = np.array(config['box_dim']) / 2.

scene_dict["sensor"]["cameraB"] = {
    "type": "perspective",
    "to_world": tf.look_at(
        origin=box_dim * np.array([0., 0., 15.]),
        target=box_dim * np.array([0., 0., 0.]),
        up=[0, 1, 0]
    ),
}

In [ ]:
scene = mi.load_dict(scene_dict)
img = mi.render(scene, spp=256)

plot_img(img)

### 5x5x5 Box

In [ ]:
mi.set_variant('cuda_ad_acoustic')

hist_leo = np.load('../notebooks-leo/data/RT-HIST-box-5-5-5.npy')
edc_leo  = EDC(hist_leo, db=True)

config = {
    "box_dim":     [5., 5., 5.],
    "mic_pos":     [2., 2., 1.],
    "speaker_pos": [1., 1., 1.],
    "speaker_radius": 0.1,

    "absorption": [(1, 0.1), (2, 0.2), (3, 0.5), (4, 0.7), (5, 0.9)],
    "scattering": 0.0,

    "wav_bins": 5,      # x
    "time_bins": 2000,  # y
    "max_time": 2,

    "integrator": "prb_acoustic",
    "max_depth": 150,
    "spp": 2**22, # 2**14 * 2048 = 2**13 * 2**11 = 2^12 * ~time_bins
}

scene_dict = shoebox_scene(**config)
scene = mi.load_dict(scene_dict)

data = np.array(mi.render(scene))
hist = data[:, :, 0] / config["spp"]
cnts = data[:, :, 1]
edc  = EDC(hist, db=True)

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, sharex=True, figsize=(12, 6))
ax1.plot(hist)
ax1.set_ylim(0., 0.00045)
ax1.set_title("Histogram")

ax2.plot(edc)
ax2.set_ylim(-350., 0.)
ax2.set_title("Energy [dB]")

ax3.plot(hist_leo)
ax3.set_ylim(0., 0.00045)
ax3.set_title("Histogram Leo")

ax4.plot(edc_leo)
ax4.set_ylim(-350., 0.)
ax4.set_title("Energy [dB] Leo")
plt.show()

#np.save('../data/mi3-prba-box-5-5-5.npy', hist)